In [1]:
import pandas as pd

In [2]:
original_Lux = pd.read_json('Luxury_Beauty.json', lines=True)
original_meta = pd.read_json('meta_Luxury_Beauty.json', lines=True)
prime = pd.read_json('meta_Prime_Pantry.json', lines=True)

df = original_Lux.copy()
meta = original_meta[['title', 'description', 'asin']].copy()


In [29]:
test_set_meta = meta.sample(n=2500, random_state=42)
test_set_meta = test_set_meta.reset_index(drop=True)

In [30]:
test_set_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2500 non-null   object
 1   description  2500 non-null   object
 2   asin         2500 non-null   object
dtypes: object(3)
memory usage: 58.7+ KB


In [31]:
test_set_lux = df.sample(n=114927, random_state=42)
test_set_lux = test_set_lux.reset_index(drop=True)

In [32]:
test_set_lux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114927 entries, 0 to 114926
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         114927 non-null  int64 
 1   vote            20690 non-null   object
 2   verified        114927 non-null  bool  
 3   reviewTime      114927 non-null  object
 4   reviewerID      114927 non-null  object
 5   asin            114927 non-null  object
 6   reviewerName    114921 non-null  object
 7   reviewText      114846 non-null  object
 8   summary         114891 non-null  object
 9   unixReviewTime  114927 non-null  int64 
 10  style           50267 non-null   object
 11  image           1491 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 9.8+ MB


In [33]:
test_set_lux.to_json('test_Luxury_Beauty.json',orient='records', lines=True)

# original_meta = test_set_meta

In [34]:
test_set_meta.to_json('test_meta_Luxury_Beauty.json',orient='records', lines=True)

In [35]:
import pandas as pd
shit = pd.read_json('Luxury_Beauty.json', lines=True)

In [36]:
shit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574628 entries, 0 to 574627
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         574628 non-null  int64 
 1   vote            103689 non-null  object
 2   verified        574628 non-null  bool  
 3   reviewTime      574628 non-null  object
 4   reviewerID      574628 non-null  object
 5   asin            574628 non-null  object
 6   reviewerName    574597 non-null  object
 7   reviewText      574228 non-null  object
 8   summary         574445 non-null  object
 9   unixReviewTime  574628 non-null  int64 
 10  style           251013 non-null  object
 11  image           7418 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 48.8+ MB


In [37]:
chooth = pd.read_json('test_Luxury_Beauty.json', lines=True)
maadar = pd.read_json('test_meta_Luxury_Beauty.json', lines=True)


In [38]:
chooth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114927 entries, 0 to 114926
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         114927 non-null  int64 
 1   vote            20690 non-null   object
 2   verified        114927 non-null  bool  
 3   reviewTime      114927 non-null  object
 4   reviewerID      114927 non-null  object
 5   asin            114927 non-null  object
 6   reviewerName    114921 non-null  object
 7   reviewText      114846 non-null  object
 8   summary         114891 non-null  object
 9   unixReviewTime  114927 non-null  int64 
 10  style           50267 non-null   object
 11  image           1491 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 9.8+ MB


In [39]:
maadar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2500 non-null   object
 1   description  2500 non-null   object
 2   asin         2500 non-null   object
dtypes: object(3)
memory usage: 58.7+ KB


In [6]:
import pandas as pd
import pickle

test = pd.read_json('test_meta_Luxury_Beauty.json', lines=True)
with open('Luxury_Beauty_text_name_dict.json', 'rb') as f:
    name_dict = pickle.load(f)
df = pd.DataFrame(name_dict)

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2500 non-null   object
 1   description  2500 non-null   object
 2   asin         2500 non-null   object
dtypes: object(3)
memory usage: 58.7+ KB


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 427 entries, 1 to 2005
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        427 non-null    object
 1   description  427 non-null    object
dtypes: object(2)
memory usage: 10.0+ KB


In [1]:
import os
import os.path
import gzip
import json
import pickle
from tqdm import tqdm
from collections import defaultdict

def parse(path):
    g = gzip.open(path, 'rb')
    for l in tqdm(g):
        yield json.loads(l)

def preprocess(fname):
    countU = defaultdict(lambda: 0) #counting how many reviews each user made ( reviewerID)
    countP = defaultdict(lambda: 0) # counting how many reviews each product (asin) received.
    line = 0

    file_path = f'{fname}.json.gz' # data over here

    # counting interactions for each user and item
    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']
        countU[rev] += 1
        countP[asin] += 1

    usermap = dict()
    usernum = 0
    itemmap = dict()
    itemnum = 0
    User = dict()
    review_dict = {}
    name_dict = {'title':{}, 'description':{}}

    f = open(f'meta_{fname}.json', 'r')
    json_data = f.readlines()
    f.close()
    data_list = [json.loads(line[:-1]) for line in json_data]
    meta_dict = {}
    for l in data_list:
        meta_dict[l['asin']] = l

    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']

        # Added Magazine over here
        threshold = 1
        if ('Beauty' in fname) or ('Toys' in fname) or ('Magazine_Subscriptions' in fname):
            # Changed threshold to 3 from 4  since the data is small of magazine
            threshold = 3

        if countU[rev] < threshold or countP[asin] < threshold:
            continue

        if rev in usermap:
            userid = usermap[rev]
        else:
            usernum += 1
            userid = usernum
            usermap[rev] = userid
            User[userid] = []

        if asin in itemmap:
            itemid = itemmap[asin]
        else:
            itemnum += 1
            itemid = itemnum
            itemmap[asin] = itemid
        User[userid].append([time, itemid])


        if itemmap[asin] in review_dict:
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except:
                a = 0
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except:
                a = 0
        else:
            review_dict[itemmap[asin]] = {'review': {}, 'summary':{}}
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except:
                a = 0
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except:
                a = 0
        try:
            if len(meta_dict[asin]['description']) ==0:
                name_dict['description'][itemmap[asin]] = 'Empty description'
            else:
                name_dict['description'][itemmap[asin]] = meta_dict[asin]['description'][0]
            name_dict['title'][itemmap[asin]] = meta_dict[asin]['title']
        except:
            a =0

    with open(f'{fname}_draft_69.json', 'wb') as tf:
        pickle.dump(name_dict, tf)

    for userid in User.keys():
        User[userid].sort(key=lambda x: x[0])

    print(usernum, itemnum)

    f = open(f'{fname}.txt', 'w') # data over here
    for user in User.keys():
        for i in User[user]:
            f.write('%d %d\n' % (user, i[1]))
    f.close()

In [2]:
preprocess('Luxury_Beauty')

114927it [00:00, 233948.07it/s]
114927it [00:00, 241837.65it/s]

1514 2008


In [4]:
import pandas as pd

with open('Luxury_Beauty_draft_69.json', 'rb') as f:
    temp = pickle.load(f)
df2 = pd.DataFrame(temp)

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2007 entries, 1 to 2008
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2007 non-null   object
 1   description  2007 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [9]:
import os
import os.path
import gzip
import json
import pickle
from tqdm import tqdm
from collections import defaultdict


def parse(path):
    g = gzip.open(path, 'rb')
    for l in tqdm(g):
        yield json.loads(l)


def preprocess(fname):
    countU = defaultdict(lambda: 0)
    countP = defaultdict(lambda: 0)
    line = 0

    file_path = f'{fname}.json.gz'  # review data file

    # Counting interactions for each user and item
    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']
        countU[rev] += 1
        countP[asin] += 1

    usermap = dict()
    usernum = 0
    itemmap = dict()  # maps ASIN to an initial (possibly non-sequential) ID
    itemnum = 0
    User = dict()
    review_dict = {}
    name_dict = {'title': {}, 'description': {}}

    # Load raw meta data
    with open(f'meta_{fname}.json', 'r') as f:
        json_data = f.readlines()
    data_list = [json.loads(line.strip()) for line in json_data]
    meta_dict = {}
    for l in data_list:
        meta_dict[l['asin']] = l

    # Process the review data
    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']

        # Set threshold for filtering interactions
        threshold = 5
        if ('Beauty' in fname) or ('Toys' in fname) or ('Magazine_Subscriptions' in fname):
            threshold = 3

        if countU[rev] < threshold or countP[asin] < threshold:
            continue

        # Map reviewer to a new integer ID
        if rev in usermap:
            userid = usermap[rev]
        else:
            usernum += 1
            userid = usernum
            usermap[rev] = userid
            User[userid] = []

        # Map product ASIN to an initial item ID
        if asin in itemmap:
            itemid = itemmap[asin]
        else:
            itemnum += 1
            itemid = itemnum
            itemmap[asin] = itemid
        User[userid].append([time, itemid])

        # Build review_dict (optional) and name_dict with meta data
        if itemmap[asin] in review_dict:
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except Exception as e:
                pass
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except Exception as e:
                pass
        else:
            review_dict[itemmap[asin]] = {'review': {}, 'summary': {}}
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except Exception as e:
                pass
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except Exception as e:
                pass
        try:
            if len(meta_dict[asin]['description']) == 0:
                name_dict['description'][itemmap[asin]] = 'Empty description'
            else:
                name_dict['description'][itemmap[asin]] = meta_dict[asin]['description'][0]
            name_dict['title'][itemmap[asin]] = meta_dict[asin]['title']
        except Exception as e:
            pass

    # --- Re-indexing step ---
    # Create new dictionaries with sequential keys and a mapping from old to new IDs.
    new_title = {}
    new_description = {}
    new_itemmap = {}  # maps old item id to new sequential id
    new_index = 1
    # Sort keys for reproducibility; alternatively, you could iterate in any order.
    for old_key in sorted(name_dict['title'].keys()):
        new_itemmap[old_key] = new_index
        new_title[new_index] = name_dict['title'][old_key]
        new_description[new_index] = name_dict['description'][old_key]
        new_index += 1
    # Replace name_dict with re-indexed dictionaries
    name_dict['title'] = new_title
    name_dict['description'] = new_description
    # Update the total number of items to reflect the new indexing
    new_itemnum = len(new_title)

    # Update the User interactions: remap item IDs using new_itemmap.
    for userid in User.keys():
        for i in range(len(User[userid])):
            old_itemid = User[userid][i][1]
            if old_itemid in new_itemmap:
                User[userid][i][1] = new_itemmap[old_itemid]

    # Save the re-indexed meta data file
    with open(f'{fname}spidey.json.gz', 'wb') as tf:
        pickle.dump(name_dict, tf)

    for userid in User.keys():
        User[userid].sort(key=lambda x: x[0])

    print("User count:", usernum, "Original item count:", itemnum, "Re-indexed item count:", new_itemnum)

    # Write user-item interaction pairs using the new indexing
    with open(f'{fname}.txt', 'w') as f:
        for user in User.keys():
            for i in User[user]:
                f.write('%d %d\n' % (user, i[1]))


In [10]:
preprocess('Luxury_Beauty')

114927it [00:00, 234837.07it/s]
114927it [00:00, 242457.77it/s]

User count: 1514 Original item count: 2008 Re-indexed item count: 2007


In [14]:
with open('Luxury_Beautyspidey.json.gz', 'rb') as f:
    temp = pickle.load(f)
df2 = pd.DataFrame(temp)


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2007 entries, 1 to 2007
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2007 non-null   object
 1   description  2007 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [7]:
import os
import os.path
import gzip
import json
import pickle
from tqdm import tqdm
from collections import defaultdict


def parse(path):
    g = gzip.open(path, 'rb')
    for l in tqdm(g):
        yield json.loads(l)


def preprocess(fname):
    countU = defaultdict(lambda: 0)
    countP = defaultdict(lambda: 0)
    line = 0

    file_path = f'{fname}.json.gz'  # review data file

    # Counting interactions for each user and item
    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']
        countU[rev] += 1
        countP[asin] += 1

    usermap = dict()
    usernum = 0
    itemmap = dict()  # maps ASIN to an initial (possibly non-sequential) ID
    itemnum = 0
    User = dict()
    review_dict = {}
    name_dict = {'title': {}, 'description': {}}

    # Load raw meta data
    with open(f'meta_{fname}.json', 'r') as f:
        json_data = f.readlines()
    data_list = [json.loads(line.strip()) for line in json_data]
    meta_dict = {}
    for l in data_list:
        meta_dict[l['asin']] = l

    # Process the review data
    for l in parse(file_path):
        line += 1
        asin = l['asin']
        rev = l['reviewerID']
        time = l['unixReviewTime']

        # Set threshold for filtering interactions
        threshold = 5
        if ('Beauty' in fname) or ('Toys' in fname) or ('Magazine_Subscriptions' in fname):
            threshold = 3

        if countU[rev] < threshold or countP[asin] < threshold:
            continue

        # Map reviewer to a new integer ID
        if rev in usermap:
            userid = usermap[rev]
        else:
            usernum += 1
            userid = usernum
            usermap[rev] = userid
            User[userid] = []

        # Map product ASIN to an initial item ID
        if asin in itemmap:
            itemid = itemmap[asin]
        else:
            itemnum += 1
            itemid = itemnum
            itemmap[asin] = itemid
        User[userid].append([time, itemid])

        # Build review_dict (optional) and name_dict with meta data
        if itemmap[asin] in review_dict:
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except Exception as e:
                pass
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except Exception as e:
                pass
        else:
            review_dict[itemmap[asin]] = {'review': {}, 'summary': {}}
            try:
                review_dict[itemmap[asin]]['review'][usermap[rev]] = l['reviewText']
            except Exception as e:
                pass
            try:
                review_dict[itemmap[asin]]['summary'][usermap[rev]] = l['summary']
            except Exception as e:
                pass
        try:
            if len(meta_dict[asin]['description']) == 0:
                name_dict['description'][itemmap[asin]] = 'Empty description'
            else:
                name_dict['description'][itemmap[asin]] = meta_dict[asin]['description'][0]
            name_dict['title'][itemmap[asin]] = meta_dict[asin]['title']
        except Exception as e:
            pass

    # --- Re-indexing step ---
    # Create new dictionaries with sequential keys and a mapping from old to new IDs.
    new_title = {}
    new_description = {}
    new_itemmap = {}  # maps old item id to new sequential id
    new_index = 1
    # Sort keys for reproducibility; alternatively, you could iterate in any order.
    for old_key in sorted(name_dict['title'].keys()):
        new_itemmap[old_key] = new_index
        new_title[new_index] = name_dict['title'][old_key]
        new_description[new_index] = name_dict['description'][old_key]
        new_index += 1
    # Replace name_dict with re-indexed dictionaries
    name_dict['title'] = new_title
    name_dict['description'] = new_description
    # Update the total number of items to reflect the new indexing
    new_itemnum = len(new_title)

    # Update the User interactions: remap item IDs using new_itemmap.
    for userid in User.keys():
        for i in range(len(User[userid])):
            old_itemid = User[userid][i][1]
            if old_itemid in new_itemmap:
                User[userid][i][1] = new_itemmap[old_itemid]

    # Save the re-indexed meta data file
    with open(f'{fname}_small_check.gz', 'wb') as tf:
        pickle.dump(name_dict, tf)

    for userid in User.keys():
        User[userid].sort(key=lambda x: x[0])

    print("User count:", usernum, "Original item count:", itemnum, "Re-indexed item count:", new_itemnum)

    # Write user-item interaction pairs using the new indexing
    with open(f'{fname}.txt', 'w') as f:
        for user in User.keys():
            for i in User[user]:
                f.write('%d %d\n' % (user, i[1]))


In [8]:
preprocess('meta_daredevil')

FileNotFoundError: [Errno 2] No such file or directory: 'meta_daredevil.json.gz'